In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import joblib
from pathlib import Path
import time

#xgboost
from xgboost import XGBClassifier

# sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score, RandomizedSearchCV, StratifiedKFold


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score
from sklearn.base import BaseEstimator, TransformerMixin, clone

from scipy.stats import ttest_rel
from scipy.stats import randint, uniform

#hiperparamentros search
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
from scipy.stats import randint, uniform,loguniform


# Importações locais
from setup_notebook import setup_path
setup_path()
from src.model_utils import *
#from src.preprocess_utils_diab2 import *
from src.preprocess_utils_diab14 import *


BASE = Path.cwd().parent
DATA_DIR = BASE/"data"/"raw"
X       = pd.read_csv(DATA_DIR/"train.csv")
X_train = pd.read_csv(DATA_DIR/"X_train_raw.csv")
X_test  = pd.read_csv(DATA_DIR/"X_test_raw.csv")
y_train = pd.read_csv(DATA_DIR/"y_train_raw.csv").values.ravel()
y_test  = pd.read_csv(DATA_DIR/"y_test_raw.csv").values.ravel()
print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))




#Processo finalizado em: 21:25:36


In [2]:
#função basica
def vizualizar_transformacao(PP,X):
    dfX = PP.transform(X)

    
    print("═" * 70)
    print("DETALHAMENTO DAS NOVAS COLUNAS CRIADAS")
    print("═" * 70)

# Identificar novas colunas
    original_cols = X.columns.tolist() if hasattr(X, 'columns') else []
    transformed_cols = dfX.columns.tolist() if hasattr(dfX, 'columns') else []
    new_cols = [col for col in transformed_cols if col not in original_cols]

    print(f"\n📈 RESUMO ESTATÍSTICO:")
    print(f"   • Dimensão originais: {X.shape}")
    print(f"   • Colunas originais: {len(original_cols)}")
    print(f"   • Colunas após transformação: {len(transformed_cols)}")
    print(f"   • Novas colunas criadas: {len(new_cols)}")
    print(f"   • Aumento: {(len(new_cols)/len(original_cols)*100):.1f}%")

    if new_cols:
        print(f"\n📋 LISTA DAS NOVAS COLUNAS ({len(new_cols)} no total):")
        print("-" * 50)
        
        # Exibir em colunas para melhor visualização
        for i in range(0, len(new_cols), 3):
            line_cols = new_cols[i:i+3]
            col_str = "   ".join([f"{col:<25}" for col in line_cols])
            print(f"   {col_str}")
        
        print(f"\n🔍 DETALHES DAS NOVAS COLUNAS:")
        print("-" * 50)
        
        for col in new_cols[:len(new_cols)]:
            unique_vals = dfX[col].nunique() if hasattr(dfX[col], 'nunique') else 'N/A'
            dtype = dfX[col].dtype
            non_null = dfX[col].count() if hasattr(dfX[col], 'count') else 'N/A'
            total = len(dfX)
            
            print(f"   • {col} {dtype}")
            print(f"      Valores únicos: {unique_vals}")
            print(f"      Não nulos: {non_null}/{total} ({(non_null/total*100):.1f}%)")
            
            # Mostrar valores únicos se forem poucos
            if hasattr(dfX[col], 'unique') and unique_vals <= 5:
                vals = dfX[col].unique()[:5]
                print(f"     Valores: {vals}")
            print()
        
    else:
        print("\nℹ️  Nenhuma nova coluna criada - transformações aplicadas in-place.")
    
    print(f"\n👁️  VISUALIZAÇÃO (primeiras 5 linhas):")
    display(dfX.head(5))
    return dfX

In [3]:
X.head(5)

,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,...,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history,diagnosed_diabetes
0,0,31,1,45,7.7,6.8,6.1,33.4,0.93,112,...,Female,Hispanic,Highschool,Lower-Middle,Current,Employed,0,0,0,1.0
1,1,50,2,73,5.7,6.5,5.8,23.8,0.83,120,...,Female,White,Highschool,Upper-Middle,Never,Employed,0,0,0,1.0
2,2,32,3,158,8.5,7.4,9.1,24.1,0.83,95,...,Male,Hispanic,Highschool,Lower-Middle,Never,Retired,0,0,0,0.0
3,3,54,3,77,4.6,7.0,9.2,26.6,0.83,121,...,Female,White,Highschool,Lower-Middle,Current,Employed,0,1,0,1.0
4,4,54,1,55,5.7,6.2,5.1,28.8,0.90,108,...,Male,White,Highschool,Upper-Middle,Never,Retired,0,1,0,1.0


In [4]:
BASE = Path.cwd().parent

PP1 = joblib.load(BASE/'src'/'preprocess_diabetes_v1.4.joblib')['preprocessador']
out1=vizualizar_transformacao(PP1,X)


══════════════════════════════════════════════════════════════════════
DETALHAMENTO DAS NOVAS COLUNAS CRIADAS
══════════════════════════════════════════════════════════════════════

📈 RESUMO ESTATÍSTICO:
   • Dimensão originais: (700000, 26)
   • Colunas originais: 26
   • Colunas após transformação: 24
   • Novas colunas criadas: 0
   • Aumento: 0.0%

ℹ️  Nenhuma nova coluna criada - transformações aplicadas in-place.

👁️  VISUALIZAÇÃO (primeiras 5 linhas):


,age,alcohol_consumption_per_week,bmi,cholesterol_total,diastolic_bp,diet_score,hdl_cholesterol,heart_rate,ldl_cholesterol,physical_activity_minutes_per_week,...,waist_to_hip_ratio,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history
0,31.0,1.0,33.400002,199.0,70.0,7.7,58.0,62.0,114.0,45.0,...,0.93,Female,Hispanic,Highschool,Lower-Middle,Current,Employed,0,0,0
1,50.0,2.0,23.799999,199.0,77.0,5.7,50.0,71.0,121.0,73.0,...,0.83,Female,White,Highschool,Upper-Middle,Never,Employed,0,0,0
2,32.0,3.0,24.100000,188.0,89.0,8.5,59.0,73.0,114.0,158.0,...,0.83,Male,Hispanic,Highschool,Lower-Middle,Never,Retired,0,0,0
3,54.0,3.0,26.600000,182.0,69.0,4.6,54.0,74.0,85.0,77.0,...,0.83,Female,White,Highschool,Lower-Middle,Current,Employed,0,1,0
4,54.0,1.0,28.799999,206.0,60.0,5.7,49.0,85.0,131.0,55.0,...,0.90,Male,White,Highschool,Upper-Middle,Never,Retired,0,1,0


In [5]:
from src.preprocess_utils_diab2 import *

PP2 = joblib.load(BASE/'src'/'preprocess_diabetes_v2.joblib')['preprocessador']
out2=vizualizar_transformacao(PP2,X)

══════════════════════════════════════════════════════════════════════
DETALHAMENTO DAS NOVAS COLUNAS CRIADAS
══════════════════════════════════════════════════════════════════════

📈 RESUMO ESTATÍSTICO:
   • Dimensão originais: (700000, 26)
   • Colunas originais: 26
   • Colunas após transformação: 33
   • Novas colunas criadas: 9
   • Aumento: 34.6%

📋 LISTA DAS NOVAS COLUNAS (9 no total):
--------------------------------------------------
   continuous_risk_score       global_prob_risk            risk_age_cont            
   risk_age_p                  risk_fh_p                   risk_paw_cont            
   risk_paw_p                  Age_Group                   PAW_Group                

🔍 DETALHES DAS NOVAS COLUNAS:
--------------------------------------------------
   • continuous_risk_score float32
      Valores únicos: 30914
      Não nulos: 700000/700000 (100.0%)

   • global_prob_risk float32
      Valores únicos: 48
      Não nulos: 700000/700000 (100.0%)

   • risk_age_co

,age,alcohol_consumption_per_week,bmi,cholesterol_total,continuous_risk_score,diastolic_bp,diet_score,global_prob_risk,hdl_cholesterol,heart_rate,...,ethnicity,education_level,income_level,smoking_status,employment_status,Age_Group,PAW_Group,family_history_diabetes,hypertension_history,cardiovascular_history
0,31.0,1.0,33.400002,199.0,0.084485,70.0,7.7,0.588333,58.0,62.0,...,Hispanic,Highschool,Lower-Middle,Current,Employed,Adulto (30–44),Baixo (<150),0,0,0
1,50.0,2.0,23.799999,199.0,0.173410,77.0,5.7,0.614367,50.0,71.0,...,White,Highschool,Upper-Middle,Never,Employed,Meia-idade (45–59),Baixo (<150),0,0,0
2,32.0,3.0,24.100000,188.0,0.075634,89.0,8.5,0.512100,59.0,73.0,...,Hispanic,Highschool,Lower-Middle,Never,Retired,Adulto (30–44),Recomendado (150-300),0,0,0
3,54.0,3.0,26.600000,182.0,0.192804,69.0,4.6,0.614367,54.0,74.0,...,White,Highschool,Lower-Middle,Current,Employed,Meia-idade (45–59),Baixo (<150),0,1,0
4,54.0,1.0,28.799999,206.0,0.196832,60.0,5.7,0.614367,49.0,85.0,...,White,Highschool,Upper-Middle,Never,Retired,Meia-idade (45–59),Baixo (<150),0,1,0


In [6]:
from src.preprocess_utils_diab3 import *

PP3 = joblib.load(BASE/'src'/'preprocess_diabetes_v3.joblib')['pipeline']
out3=vizualizar_transformacao(PP3,X)

══════════════════════════════════════════════════════════════════════
DETALHAMENTO DAS NOVAS COLUNAS CRIADAS
══════════════════════════════════════════════════════════════════════

📈 RESUMO ESTATÍSTICO:
   • Dimensão originais: (700000, 26)
   • Colunas originais: 26
   • Colunas após transformação: 33
   • Novas colunas criadas: 9
   • Aumento: 34.6%

📋 LISTA DAS NOVAS COLUNAS (9 no total):
--------------------------------------------------
   continuous_risk_score       global_prob_risk            risk_age_cont            
   risk_age_p                  risk_fh_p                   risk_paw_cont            
   risk_paw_p                  Age_Group                   PAW_Group                

🔍 DETALHES DAS NOVAS COLUNAS:
--------------------------------------------------
   • continuous_risk_score float32
      Valores únicos: 30914
      Não nulos: 700000/700000 (100.0%)

   • global_prob_risk float32
      Valores únicos: 48
      Não nulos: 700000/700000 (100.0%)

   • risk_age_co

,age,alcohol_consumption_per_week,bmi,cholesterol_total,continuous_risk_score,diastolic_bp,diet_score,global_prob_risk,hdl_cholesterol,heart_rate,...,income_level,gender,ethnicity,smoking_status,employment_status,Age_Group,PAW_Group,family_history_diabetes,hypertension_history,cardiovascular_history
0,31.0,1.0,33.400002,199.0,0.084485,70.0,7.7,0.588708,58.0,62.0,...,1,Female,Hispanic,Current,Employed,Adulto (30–44),Baixo (<150),0,0,0
1,50.0,2.0,23.799999,199.0,0.173410,77.0,5.7,0.614308,50.0,71.0,...,3,Female,White,Never,Employed,Meia-idade (45–59),Baixo (<150),0,0,0
2,32.0,3.0,24.100000,188.0,0.075634,89.0,8.5,0.511612,59.0,73.0,...,1,Male,Hispanic,Never,Retired,Adulto (30–44),Recomendado (150-300),0,0,0
3,54.0,3.0,26.600000,182.0,0.192804,69.0,4.6,0.614308,54.0,74.0,...,1,Female,White,Current,Employed,Meia-idade (45–59),Baixo (<150),0,1,0
4,54.0,1.0,28.799999,206.0,0.196832,60.0,5.7,0.614308,49.0,85.0,...,3,Male,White,Never,Retired,Meia-idade (45–59),Baixo (<150),0,1,0


In [8]:
out3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 33 columns):
 #   Column                              Non-Null Count   Dtype   
---  ------                              --------------   -----   
 0   age                                 700000 non-null  float32 
 1   alcohol_consumption_per_week        700000 non-null  float32 
 2   bmi                                 700000 non-null  float32 
 3   cholesterol_total                   700000 non-null  float32 
 4   continuous_risk_score               700000 non-null  float32 
 5   diastolic_bp                        700000 non-null  float32 
 6   diet_score                          700000 non-null  float32 
 7   global_prob_risk                    700000 non-null  float32 
 8   hdl_cholesterol                     700000 non-null  float32 
 9   heart_rate                          700000 non-null  float32 
 10  ldl_cholesterol                     700000 non-null  float32 
 11  physical_acti